In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from utils import printMatrix
from matplotlib.animation import FuncAnimation
from scipy import stats
from scipy import signal

plt.style.use('seaborn-pastel')

In [ ]:
# Edificio de 4 pisos de 20 toneladas cada uno, con rigideces laterales de 57MN/m entre si
k = 57e6 # N/m
m = 20e3 # kg

k1, k2, k3, k4 = k, k, k, k
m1, m2, m3, m4 = m, m, m, m

# Matrices de rigidez y de masa
K = np.array([[k1+k2, -k2, 0, 0],
             [-k2, k2+k3, -k3, 0],
             [0, -k3, k3+k4, -k4],
             [0, 0, -k4, k4]])

M = np.diag((m1, m2, m3, m4))

print(('K = {}').format(K))
print(('M = {}').format(M))

In [ ]:
# Resuelvo autovectores y autovalores
A = np.linalg.inv(M)@K
lamb, eigv = np.linalg.eig(A)

In [ ]:
# convierto velocidades angulares a frecuencias
w = np.sqrt(lamb)
f = w/2/np.pi
print(('f = {}').format(f))

In [ ]:
# Matriz de autovectores (columnas)
print("eigv:")
printMatrix(eigv)

In [ ]:
# Matriz de masas modales
M_modal = np.transpose(eigv)@M@eigv
M_modal[M_modal<1e-3] = np.nan
print("M_modal:")
printMatrix(M_modal)

In [ ]:
# Matriz de rigideces modales
K_modal = np.transpose(eigv)@K@eigv
K_modal[K_modal<1e-3] = np.nan
print("K_modal:")
printMatrix(K_norm)

In [ ]:
# Verifico la validez de las matrices obteniendo las fercuencias modales nuevamente, ahora como w_i = sqrt(k_i/m_i)
w_2 = np.zeros(len(w))
for mode in range(len(w)):
    w_2[mode] = np.sqrt(K_modal[mode,mode]/M_modal[mode,mode])
f_2 = w_2/np.pi/2
print(('f2 = {}').format(f_2))

In [ ]:
# Normalizo vectores modales
eigv_norm = eigv
for mode in range(len(w)):
    eigv_norm[:,mode] = eigv[:,mode]/max(abs(eigv[:,mode]))
print("eigv_norm:")
printMatrix(eigv_norm)

In [ ]:
fig, ax = plt.subplots()
fig.add_axes()
ax.set(xlim=[-1,1], ylim=[0,5], )
ax.grid()
ax.set_aspect('equal')
eigv_plot = np.zeros((eigv.shape[1] + 1, eigv.shape[1]))
eigv_plot[1:, :] = eigv

ax.plot(np.zeros(eigv.shape[1] + 1),np.linspace(0,4,5), color='k', marker='o')
for col in range(eigv.shape[1]):
    ax.plot(eigv_plot[:,col],[0, 1, 2, 3, 4], marker='o')
    
plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.add_axes()
ax.set(xlim=[-1,1], ylim=[0,5], )
ax.grid()
ax.set_aspect('equal')

line1, = ax.plot([], [], lw=3)
line2, = ax.plot([], [], lw=3)
line3, = ax.plot([], [], lw=3)
line4, = ax.plot([], [], lw=3)

def init():
    line1.set_data([], [])
    line2.set_data([], [])
    line3.set_data([], [])
    line4.set_data([], [])
    return line1, line2, line3, line4

def animate(i):
    y = np.linspace(0, 4, 5)
    line1.set_data(eigv_plot[:,0]*np.sin(0.2*np.pi*i), y)
    line2.set_data(eigv_plot[:,1]*np.sin(0.2*np.pi*i), y)
    line3.set_data(eigv_plot[:,2]*np.sin(0.2*np.pi*i), y)
    line4.set_data(eigv_plot[:,3]*np.sin(0.2*np.pi*i), y)
    return line1, line2, line3, line4

anim = FuncAnimation(fig, animate, init_func=init,
                               frames=10, interval=200, blit=True)

anim.save('sine_wave.gif', writer='imagemagick')

In [ ]:
# Aceleracion maxima en funcion del tiempo
def max_accel(t,t_end):
    a = 0.5*(1-np.cos(2*np.pi*t/t_end))
    return a

In [ ]:
# Simulacion de aceleracion en forma de ruido blanco bajo la curva de aceleracion maxima
t_end = 600
delta_t = 1/10
t = np.linspace(0, t_end, t_end/delta_t)
accel = np.zeros(len(t))
X = np.zeros(len(t))
for step in range(len(t)):
    uniform_num = stats.uniform.rvs() # numero entre 0 y 1, uniforme
    X[step] = stats.norm.ppf(uniform_num) # numero tal que la probabilidad de ser menor a el en una nomal sea uniform_num
X_mean = np.mean(X)
accel = (X - X_mean)*max_accel(t,t_end) # aceleracion escalada por factor r

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
fig.add_axes()
ax.plot(t, accel, color='k', marker='')
ax.plot(t, max_accel(t,t_end), color='r')

In [ ]:
plt.psd(accel)
plt.show()

In [ ]:
f, psd = signal.welch(accel)
plt.figure()
plt.semilogy(f, np.sqrt(psd))
plt.xlabel('frequency [Hz]')
plt.ylabel('ASD [g²/Hz]')
plt.show()